# Applying XID+CIGALE to Extreme Starbursts
In this notebook, we read in the data files and prepare them for fitting with XID+CIGALE, the SED prior model extension to XID+. Here we focus on sources in [Rowan-Robinson et al. 2018](https://arxiv.org/abs/1704.07783) and claimed to have a star formation rate of $> 10^{3}\mathrm{M_{\odot}yr^{-1}}$

In [1]:

field=['Lockman-SWIRE']
#path_to_SPIRE=['/Volumes/pdh_storage/dmu_products/dmu19/dmu19_HELP-SPIRE-maps/data/']
path_to_SPIRE=['/mnt/lustre/users/astro/pdh21/Extreme_submm_SB_gals/data/']
#redshift_file=["/Volumes/pdh_storage/dmu_products/dmu24/dmu24_Lockman-SWIRE/data/master_catalogue_Lockman-SWIRE_20170710_photoz_20170802_r_and_irac1_optimised_UPDATED_IDs_20180219.fits"]
redshift_file=["/mnt/lustre/users/astro/pdh21/Extreme_submm_SB_gals/data/master_catalogue_Lockman-SWIRE_20170710_photoz_20170802_r_and_irac1_optimised_UPDATED_IDs_20180219.fits"]

In [2]:
# Parameters
field = ["XMM-LSS"]
path_to_SPIRE = ["/mnt/lustre/users/astro/pdh21/Extreme_submm_SB_gals/data/"]
redshift_file = [
    "/mnt/lustre/users/astro/pdh21/Extreme_submm_SB_gals/data/master_catalogue_xmm-lss_20180221_photoz_20180518_r_optimised.fits"
]


Import required modules

In [3]:
from astropy.io import ascii, fits
from astropy.table import Table
import pylab as plt
%matplotlib inline
from astropy import wcs


import numpy as np
import xidplus
from xidplus import moc_routines
import pickle
import os


In [4]:
try:
    source=[np.int(os.environ['SGE_TASK_ID'])-1]
except:
    source=[0]

Set image and catalogue filenames

In [5]:
#Folder containing maps
pswfits=path_to_SPIRE[0]+'{}_SPIRE250_v1.0.fits'.format(field[0])#SPIRE 250 map
pmwfits=path_to_SPIRE[0]+'{}_SPIRE350_v1.0.fits'.format(field[0])#SPIRE 350 map
plwfits=path_to_SPIRE[0]+'{}_SPIRE500_v1.0.fits'.format(field[0])#SPIRE 500 map




#output folder
output_folder='./'

Load in images, noise maps, header info and WCS information

In [6]:
#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[3].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[3].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header 
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[3].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()

XID+ uses Multi Order Coverage (MOC) maps for cutting down maps and catalogues so they cover the same area. It can also take in MOCs as selection functions to carry out additional cuts. Lets use the python module [pymoc](http://pymoc.readthedocs.io/en/latest/) to create a MOC, centered on a specific position we are interested in. We will use a HEALPix order of 15 (the resolution: higher order means higher resolution)

Read in the source we are interested in from Rowan-Robinsons's catalogue.

In [7]:
#ysb=Table.read('../../../data/MRRfiles/fromPaper.txt', format='ascii')
esb=Table.read('../../../data/MRR2018_tables/{}_sources.csv'.format(field[0]),format='ascii')


In [8]:
esb

RA,Dec,i,S 24 (μJy),S 250 (mJy),S 350 (mJy),S 500 (mJy),Z phot,Type,Chi,n bands,Z subm,Z comb,M * log(M sol.),SFR log(M sol. yr−1),field,help_id,status
float64,float64,str9,float64,float64,float64,float64,float64,str3,float64,int64,float64,float64,float64,float64,str7,str30,str50
35.28232,-4.149,g = 25.31,636.1,53.7,58.3,55.1,3.27,Sab,2.4,4,3.84,2.8,11.28,3.7,XMM-LSS,------------------------------,--------------------------------------------------
35.28037,-4.14839,--,--,--,--,--,2.55,--,--,--,--,--,--,3.59,XMM-LSS,------------------------------,--------------------------------------------------
36.59817,-4.56164,25.11,389.5,59.3,67.2,54.7,3.88,Sab,2.1,5,3.53,3.68,11.49,3.85,XMM-LSS,------------------------------,--------------------------------------------------
36.71948,-3.96377,25.18,322.8,43.4,42.8,32.1,3.57,Sbc,5.2,5,3.1,3.27,12.21,3.7,XMM-LSS,------------------------------,--------------------------------------------------
35.73369,-5.62305,23.42,422.0,73.1,60.9,42.1,4.11,Sbc,4.1,6,2.61,2.8,12.38,4.1,XMM-LSS,------------------------------,--------------------------------------------------
36.10986,-4.45889,24.59,828.6,88.4,89.5,67.0,3.92,Sab,2.0,5,3.14,3.27,12.26,4.21,XMM-LSS,------------------------------,--------------------------------------------------
36.65871,-4.14628,24.91,288.8,46.1,45.7,48.1,4.4,Sbc,2.1,6,3.92,4.01,12.2,4.03,XMM-LSS,------------------------------,--------------------------------------------------
33.711,-4.17344,22.53,1016.3,96.2,69.0,32.5,1.47,Scd,9.3,6,1.92,1.57,11.8,3.81,XMM-LSS,------------------------------,--------------------------------------------------
35.95578,-5.08144,23.85,1002.1,71.2,77.6,59.5,3.15,Sab,7.3,5,3.32,3.27,11.91,3.8,XMM-LSS,------------------------------,--------------------------------------------------


In [9]:
from astropy.coordinates import SkyCoord
from astropy import units as u
import os

c = SkyCoord(ra=[esb['RA'][source[0]]]*u.degree, dec=[esb['Dec'][source[0]]]*u.degree)

import pymoc
moc=pymoc.util.catalog.catalog_to_moc(c,100,15)

Load in catalogue you want to fit (and make any cuts). Here we use HELP's VO database and directly call it using PyVO

In [10]:
import pyvo as vo
service = vo.dal.TAPService("https://herschel-vos.phys.sussex.ac.uk/__system__/tap/run/tap")

In [11]:
resultset = service.search("SELECT TOP 10000 * FROM herschelhelp.main WHERE 1=CONTAINS(POINT('ICRS', ra, dec),CIRCLE('ICRS',"+str(c.ra.deg[0])+", "+str(c.dec.deg[0])+", 0.028 ))")

In [12]:
masterlist=resultset.table

In [13]:
masterlist

<Table length=1251>
 field            help_id           ... flag_optnir_obs flag_optnir_det
                                    ...                                
 object            object           ...      int16           int16     
------- --------------------------- ... --------------- ---------------
XMM-LSS HELP_J022101.945-040955.389 ...               7               1
XMM-LSS HELP_J022101.990-040953.830 ...               7               0
XMM-LSS HELP_J022102.259-040957.935 ...               7               7
XMM-LSS HELP_J022102.721-040958.324 ...               7               3
XMM-LSS HELP_J022102.611-040957.219 ...               7               1
XMM-LSS HELP_J022102.722-040954.674 ...               7               7
XMM-LSS HELP_J022101.784-040948.802 ...               7               7
XMM-LSS HELP_J022102.071-040942.578 ...               7               3
XMM-LSS HELP_J022102.742-040943.823 ...               7               1
    ...                         ... ...             ...             ...
XMM-LSS HELP_J022112.318-040802.930 ...               7               0
XMM-LSS HELP_J022111.762-040757.977 ...               7               1
XMM-LSS HELP_J022112.169-040802.168 ...               7               0
XMM-LSS HELP_J022112.720-040801.652 ...               7               1
XMM-LSS HELP_J022112.644-040758.932 ...               7               1
XMM-LSS HELP_J022112.384-040754.163 ...               7               1
XMM-LSS HELP_J022113.067-040811.273 ...               7               7
XMM-LSS HELP_J022113.150-040807.664 ...               7               1
XMM-LSS HELP_J022111.885-040746.278 ...               7               0
XMM-LSS HELP_J022112.535-040751.463 ...               7               7

## Filter table
We will only use objects that are not stars (i.e. have `flag_gaia` = 3)

In [14]:
prior_list=masterlist[(masterlist['flag_gaia']!=3) & (masterlist['flag_optnir_det']>=3)]

### Check original source in prior list, if not then add

In [15]:
in_prior=prior_list['help_id']==masterlist[np.argmin(c.separation(SkyCoord(masterlist['ra'],masterlist['dec'])).arcsec)]['help_id']
if in_prior.sum()==1:
    print('in prior list')
else:
    from astropy.table import vstack,hstack
    prior_list=vstack((prior_list,masterlist[masterlist['help_id']==masterlist[np.argmin(c.separation(SkyCoord(masterlist['ra'],masterlist['dec'])).arcsec)]['help_id']]))

Having identified which masterlist source corresponds to the candidate extreme starburst, update web table

In [16]:

esb_web=Table.read('../../../data/MRR2018_tables/{}_web.csv'.format(field[0]),format='ascii')

esb_web['help_id'][source[0]]=masterlist[np.argmin(c.separation(SkyCoord(masterlist['ra'],masterlist['dec'])).arcsec)]['help_id']
esb_web.write('../../../data/MRR2018_tables/{}_web.csv'.format(field[0]))

## Get Redshift and Uncertianty


Ken Duncan defines a median and a hierarchical bayes combination redshift. We need uncertianty so lets match via `help_id`

In [17]:
photoz=Table.read(redshift_file[0])

In [18]:
#help_id=np.empty((len(photoz)),dtype=np.dtype('U27'))
for i in range(0,len(photoz)):
    photoz['help_id'][i]=str(photoz['help_id'][i].strip()).encode('utf-8')
#photoz['help_id']=help_id

In [19]:
from astropy.table import Column, MaskedColumn
prior_list['redshift']=MaskedColumn(np.full((len(prior_list)),fill_value=0.1),mask=[False]*len(prior_list))
prior_list.add_column(MaskedColumn(np.full((len(prior_list)),fill_value=2.0),mask=[False]*len(prior_list),name='redshift_unc'))

In [20]:
photoz

help_id,RA,DEC,id,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,za_hb,chi_r_eazy,chi_r_atlas,chi_r_cosmos,chi_r_stellar,stellar_type
bytes27,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes6
HELP_J020942.239-050256.966,32.4259959475648,-5.049157158297614,1,1.0759,0.371,1.8136,0.796,-99.0,-99.0,-99.0,-99.0,1.017680534872741,-99.0,-99.0,-99.0,-99.0,
HELP_J021023.321-043727.650,32.5971699475648,-4.624347158297615,3,1.5707,1.0915,2.0597,0.801,-99.0,-99.0,-99.0,-99.0,1.556384130470285,26.980099999999997,27.777672727272726,20.148027272727273,64.0746909090909,f2ii
HELP_J021002.551-043935.875,32.5106279475648,-4.659965158297615,4,1.046,0.3265,1.822,0.799,-99.0,-99.0,-99.0,-99.0,0.6908199212705971,0.22362377777777778,0.1709857777777778,0.18541488888888888,0.23163566666666668,g8i
HELP_J021021.212-044302.122,32.5883849475648,-4.717256158297614,6,0.5829,0.4776,0.6858,0.432,0.3753,0.2721,0.4732,0.359,0.5829851093325998,1.475635,1.38504,1.4375271428571428,2.7459678571428574,g8v
HELP_J021018.568-043753.069,32.577367947564795,-4.631408158297615,7,1.0459,0.3669,1.7305,0.798,-99.0,-99.0,-99.0,-99.0,0.9876858183925309,0.2791742222222222,0.4089527777777778,0.4059371111111111,1.0433302222222223,f5iii
HELP_J021007.208-043759.443,32.53003428167084,-4.633178581155195,8,1.1953,0.4127,2.0142,0.8,-99.0,-99.0,-99.0,-99.0,1.085271787314885,0.009960185,0.22611725,0.1765124,2.4687815,m0iii
HELP_J021009.556-043759.657,32.5398179475648,-4.633238158297615,9,1.0761,0.371,1.7884,0.797,-99.0,-99.0,-99.0,-99.0,1.0420018741130268,1.012204375,1.120773,1.47356375,1.66391875,f0i
HELP_J021019.712-043726.528,32.58213410002509,-4.624035574485226,11,1.0838,0.3917,1.8305,0.797,-99.0,-99.0,-99.0,-99.0,1.0298047772067913,0.35489225,0.083737625,0.688846,1.4754055,g5v
HELP_J021000.394-043800.429,32.50164195324052,-4.633452484763327,12,1.0612,0.3466,1.7884,0.796,-99.0,-99.0,-99.0,-99.0,1.09780218548486,0.098802175,0.077186125,0.57019675,0.9009085,f2iii


In [21]:
ii=0
for i in range(0,len(prior_list)):
    ind=photoz['help_id'] == prior_list['help_id'][i]
    try:
        
        prior_list['redshift'][i]=photoz['z1_median'][ind]
        prior_list['redshift_unc'][i]=np.max(np.array([np.abs(photoz['z1_median'][ind]-photoz['z1_min'][ind]),np.abs(photoz['z1_max'][ind]-photoz['z1_median'][ind])]))
        
        #prior_list['redshift_unc'].mask[i]=False
        #prior_list['redshift'].mask[i]=False

    except ValueError:
        None

In [22]:
prior_list['redshift']

0.6275
0.7286
0.5848
0.6014
0.5662
0.4897
0.591
0.8147
0.0395
0.4318
0.4679


In [23]:
prior_list[prior_list['redshift'].mask == False]

<Table length=35>
 field            help_id           ... flag_optnir_det     redshift_unc    
                                    ...                                     
 object            object           ...      int16            float64       
------- --------------------------- ... --------------- --------------------
XMM-LSS HELP_J022102.292-040900.479 ...               7  0.04919999999999991
XMM-LSS HELP_J022101.448-040827.637 ...               7  0.07290000000000008
XMM-LSS HELP_J022102.462-040820.496 ...               7  0.04139999999999999
XMM-LSS HELP_J022102.657-040750.001 ...               7 0.054299999999999904
XMM-LSS HELP_J022106.983-041030.281 ...               7  0.03589999999999993
XMM-LSS HELP_J022108.447-041020.710 ...               7  0.06430000000000002
XMM-LSS HELP_J022109.253-041019.137 ...               7  0.05480000000000007
XMM-LSS HELP_J022102.884-041000.291 ...               7  0.08809999999999996
XMM-LSS HELP_J022103.683-040949.345 ...               7               0.0458
    ...                         ... ...             ...                  ...
XMM-LSS HELP_J022105.377-040744.836 ...               7  0.04200000000000004
XMM-LSS HELP_J022110.508-040833.761 ...               7  0.04630000000000001
XMM-LSS HELP_J022110.835-040824.238 ...               7 0.058699999999999974
XMM-LSS HELP_J022111.154-040816.775 ...               7  0.06520000000000004
XMM-LSS HELP_J022111.501-040742.227 ...               7               0.0791
XMM-LSS HELP_J022107.951-040734.247 ...               7               0.4365
XMM-LSS HELP_J022108.853-040719.896 ...               7  0.22039999999999993
XMM-LSS HELP_J022112.021-040853.226 ...               3  0.07230000000000003
XMM-LSS HELP_J022113.067-040811.273 ...               7  0.06540000000000001
XMM-LSS HELP_J022107.328-040854.215 ...               7  0.33750000000000013

In [24]:
dist_matrix=np.zeros((len(prior_list),len(prior_list)))
from astropy.coordinates import SkyCoord
from astropy import units as u
for i in range(0,len(prior_list)):
    for j in range(0,len(prior_list)):
        if i>j:
            coord1 = SkyCoord(ra=prior_list['ra'][i]*u.deg,dec=prior_list['dec'][i]*u.deg,frame='icrs')

            coord2=SkyCoord(ra=prior_list['ra'][j]*u.deg,dec=prior_list['dec'][j]*u.deg)
            dist_matrix[i,j] = coord1.separation(coord2).value

In [25]:
plt.hist(np.tril(dist_matrix).flatten()*3600.0,bins=np.arange(0.00001,5,0.1));

In [26]:
ind=(np.tril(dist_matrix)<1.0/3600.0) & (np.tril(dist_matrix)>0)
xx,yy=np.meshgrid(np.arange(0,len(prior_list)),np.arange(0,len(prior_list)))
yy[ind]

array([], dtype=int64)

In [27]:
prior_list[yy[ind]]

field,help_id,ra,dec,hp_idx,ebv,redshift,zspec,zspec_qual,zspec_association_flag,stellarity,stellarity_origin,f_90prime_g,ferr_90prime_g,f_ap_90prime_g,ferr_ap_90prime_g,m_90prime_g,merr_90prime_g,m_ap_90prime_g,merr_ap_90prime_g,flag_90prime_g,f_90prime_r,ferr_90prime_r,f_ap_90prime_r,ferr_ap_90prime_r,m_90prime_r,merr_90prime_r,m_ap_90prime_r,merr_ap_90prime_r,flag_90prime_r,f_90prime_z,ferr_90prime_z,f_ap_90prime_z,ferr_ap_90prime_z,m_90prime_z,merr_90prime_z,m_ap_90prime_z,merr_ap_90prime_z,flag_90prime_z,f_acs_f435w,ferr_acs_f435w,f_ap_acs_f435w,ferr_ap_acs_f435w,m_acs_f435w,merr_acs_f435w,m_ap_acs_f435w,merr_ap_acs_f435w,flag_acs_f435w,f_acs_f606w,ferr_acs_f606w,f_ap_acs_f606w,ferr_ap_acs_f606w,m_acs_f606w,merr_acs_f606w,m_ap_acs_f606w,merr_ap_acs_f606w,flag_acs_f606w,f_acs_f775w,ferr_acs_f775w,f_ap_acs_f775w,ferr_ap_acs_f775w,m_acs_f775w,merr_acs_f775w,m_ap_acs_f775w,merr_ap_acs_f775w,flag_acs_f775w,f_acs_f814w,ferr_acs_f814w,f_ap_acs_f814w,ferr_ap_acs_f814w,m_acs_f814w,merr_acs_f814w,m_ap_acs_f814w,merr_ap_acs_f814w,flag_acs_f814w,f_acs_f850lp,ferr_acs_f850lp,f_ap_acs_f850lp,ferr_ap_acs_f850lp,m_acs_f850lp,merr_acs_f850lp,m_ap_acs_f850lp,merr_ap_acs_f850lp,flag_acs_f850lp,f_bessell_b,ferr_bessell_b,f_ap_bessell_b,ferr_ap_bessell_b,m_bessell_b,merr_bessell_b,m_ap_bessell_b,merr_ap_bessell_b,flag_bessell_b,f_bessell_i,ferr_bessell_i,f_ap_bessell_i,ferr_ap_bessell_i,m_bessell_i,merr_bessell_i,m_ap_bessell_i,merr_ap_bessell_i,flag_bessell_i,f_bessell_r,ferr_bessell_r,f_ap_bessell_r,ferr_ap_bessell_r,m_bessell_r,merr_bessell_r,m_ap_bessell_r,merr_ap_bessell_r,flag_bessell_r,f_bessell_u,ferr_bessell_u,f_ap_bessell_u,ferr_ap_bessell_u,m_bessell_u,merr_bessell_u,m_ap_bessell_u,merr_ap_bessell_u,flag_bessell_u,f_bessell_v,ferr_bessell_v,f_ap_bessell_v,ferr_ap_bessell_v,m_bessell_v,merr_bessell_v,m_ap_bessell_v,merr_ap_bessell_v,flag_bessell_v,f_cfht12k_b,ferr_cfht12k_b,f_ap_cfht12k_b,ferr_ap_cfht12k_b,m_cfht12k_b,merr_cfht12k_b,m_ap_cfht12k_b,merr_ap_cfht12k_b,flag_cfht12k_b,f_cfht12k_i,ferr_cfht12k_i,f_ap_cfht12k_i,ferr_ap_cfht12k_i,m_cfht12k_i,merr_cfht12k_i,m_ap_cfht12k_i,merr_ap_cfht12k_i,flag_cfht12k_i,f_cfht12k_r,ferr_cfht12k_r,f_ap_cfht12k_r,ferr_ap_cfht12k_r,m_cfht12k_r,merr_cfht12k_r,m_ap_cfht12k_r,merr_ap_cfht12k_r,flag_cfht12k_r,f_decam_g,ferr_decam_g,f_ap_decam_g,ferr_ap_decam_g,m_decam_g,merr_decam_g,m_ap_decam_g,merr_ap_decam_g,flag_decam_g,f_decam_i,ferr_decam_i,f_ap_decam_i,ferr_ap_decam_i,m_decam_i,merr_decam_i,m_ap_decam_i,merr_ap_decam_i,flag_decam_i,f_decam_r,ferr_decam_r,f_ap_decam_r,ferr_ap_decam_r,m_decam_r,merr_decam_r,m_ap_decam_r,merr_ap_decam_r,flag_decam_r,f_decam_y,ferr_decam_y,f_ap_decam_y,ferr_ap_decam_y,m_decam_y,merr_decam_y,m_ap_decam_y,merr_ap_decam_y,flag_decam_y,f_decam_z,ferr_decam_z,f_ap_decam_z,ferr_ap_decam_z,m_decam_z,merr_decam_z,m_ap_decam_z,merr_ap_decam_z,flag_decam_z,f_galex_fuv,ferr_galex_fuv,f_ap_galex_fuv,ferr_ap_galex_fuv,m_galex_fuv,merr_galex_fuv,m_ap_galex_fuv,merr_ap_galex_fuv,flag_galex_fuv,f_galex_nuv,ferr_galex_nuv,f_ap_galex_nuv,ferr_ap_galex_nuv,m_galex_nuv,merr_galex_nuv,m_ap_galex_nuv,merr_ap_galex_nuv,flag_galex_nuv,f_gpc1_g,ferr_gpc1_g,f_ap_gpc1_g,ferr_ap_gpc1_g,m_gpc1_g,merr_gpc1_g,m_ap_gpc1_g,merr_ap_gpc1_g,flag_gpc1_g,f_gpc1_i,ferr_gpc1_i,f_ap_gpc1_i,ferr_ap_gpc1_i,m_gpc1_i,merr_gpc1_i,m_ap_gpc1_i,merr_ap_gpc1_i,flag_gpc1_i,f_gpc1_r,ferr_gpc1_r,f_ap_gpc1_r,ferr_ap_gpc1_r,m_gpc1_r,merr_gpc1_r,m_ap_gpc1_r,merr_ap_gpc1_r,flag_gpc1_r,f_gpc1_y,ferr_gpc1_y,f_ap_gpc1_y,ferr_ap_gpc1_y,m_gpc1_y,merr_gpc1_y,m_ap_gpc1_y,merr_ap_gpc1_y,flag_gpc1_y,f_gpc1_z,ferr_gpc1_z,f_ap_gpc1_z,ferr_ap_gpc1_z,m_gpc1_z,merr_gpc1_z,m_ap_gpc1_z,merr_ap_gpc1_z,flag_gpc1_z,f_hawki_k,ferr_hawki_k,f_ap_hawki_k,ferr_ap_hawki_k,m_hawki_k,merr_hawki_k,m_ap_hawki_k,merr_ap_hawki_k,flag_hawki_k,f_irac_i1,ferr_irac_i1,f_ap_irac_i1,ferr_ap_irac_i1,m_irac_i1,merr_irac_i1,m_ap_irac_i1,merr_ap_irac_i1,flag_irac_i1,f_irac_i2,ferr_irac_i2,f_ap_irac_i2,ferr_ap_irac_i2,m_irac_i2,merr_irac_i2,m_ap_irac_i2,merr

In [28]:
prior_list['redshift'].mask[yy[ind]]=True

In [29]:

prior_list=prior_list[prior_list['redshift'].mask == False]

In [30]:
prior_list

<Table length=35>
 field            help_id           ... flag_optnir_det     redshift_unc    
                                    ...                                     
 object            object           ...      int16            float64       
------- --------------------------- ... --------------- --------------------
XMM-LSS HELP_J022102.292-040900.479 ...               7  0.04919999999999991
XMM-LSS HELP_J022101.448-040827.637 ...               7  0.07290000000000008
XMM-LSS HELP_J022102.462-040820.496 ...               7  0.04139999999999999
XMM-LSS HELP_J022102.657-040750.001 ...               7 0.054299999999999904
XMM-LSS HELP_J022106.983-041030.281 ...               7  0.03589999999999993
XMM-LSS HELP_J022108.447-041020.710 ...               7  0.06430000000000002
XMM-LSS HELP_J022109.253-041019.137 ...               7  0.05480000000000007
XMM-LSS HELP_J022102.884-041000.291 ...               7  0.08809999999999996
XMM-LSS HELP_J022103.683-040949.345 ...               7               0.0458
    ...                         ... ...             ...                  ...
XMM-LSS HELP_J022105.377-040744.836 ...               7  0.04200000000000004
XMM-LSS HELP_J022110.508-040833.761 ...               7  0.04630000000000001
XMM-LSS HELP_J022110.835-040824.238 ...               7 0.058699999999999974
XMM-LSS HELP_J022111.154-040816.775 ...               7  0.06520000000000004
XMM-LSS HELP_J022111.501-040742.227 ...               7               0.0791
XMM-LSS HELP_J022107.951-040734.247 ...               7               0.4365
XMM-LSS HELP_J022108.853-040719.896 ...               7  0.22039999999999993
XMM-LSS HELP_J022112.021-040853.226 ...               3  0.07230000000000003
XMM-LSS HELP_J022113.067-040811.273 ...               7  0.06540000000000001
XMM-LSS HELP_J022107.328-040854.215 ...               7  0.33750000000000013

XID+ is built around two python classes. A prior and posterior class. There should be a prior class for each map being fitted. It is initiated with a map, noise map, primary header and map header and can be set with a MOC. It also requires an input prior catalogue and point spread function.


In [31]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=moc)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior_list['ra'],prior_list['dec'],'photoz',ID=prior_list['help_id'])
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=moc)
prior350.prior_cat(prior_list['ra'],prior_list['dec'],'photoz',ID=prior_list['help_id'])
prior350.prior_bkg(-5.0,5)
#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=moc)
prior500.prior_cat(prior_list['ra'],prior_list['dec'],'photoz',ID=prior_list['help_id'])
prior500.prior_bkg(-5.0,5)

Set PSF. For SPIRE, the PSF can be assumed to be Gaussian with a FWHM of 18.15, 25.15, 36.3 '' for 250, 350 and 500 $\mathrm{\mu m}$ respectively. Lets use the astropy module to construct a Gaussian PSF and assign it to the three XID+ prior classes.

In [32]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

#### Add MIPS 24 data

mipsfits='/Volumes/pdh_storage/dmu_products/dmu26/data/Lockman-SWIRE/MIPS/wp4_lockman-swire_mips24_map_v1.0.fits.gz'
#-----24-------------
hdulist = fits.open(mipsfits)
im24phdu=hdulist[0].header
im24hdu=hdulist[1].header

im24=hdulist[1].data
nim24=hdulist[2].data
w_24 = wcs.WCS(hdulist[1].header)
pixsize24=3600.0*w_24.wcs.cdelt[1] #pixel size (in arcseconds)
hdulist.close()




# Point response information, at the moment its 2D Gaussian,

#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize24])
#point response function for the three bands


#Set prior classes
#---prior24--------
prior24=xidplus.prior(im24.astype(float),nim24.astype(float),im24phdu,im24hdu,moc=moc)#Initialise with map, uncertianty map, wcs info and primary header
prior24.prior_cat(prior_list['ra'],prior_list['dec'],'photoz',ID=prior_list['help_id'])#Set input catalogue
prior24.prior_bkg(0,2)#Set prior on background

##---------fit using seb's empiricall beam-----------------------
#-----24-------------
hdulist = fits.open('/Volumes/pdh_storage/dmu_products/dmu17/dmu17_HELP_Legacy_Maps/Lockman-SWIRE/data/dmu17_MIPS_Lockman-SWIRE_20171122.fits')
prf24=hdulist[1].data
centre=np.long((hdulist[1].header['NAXIS1']-1)/2)
radius=20
hdulist.close()


import scipy.ndimage
prior24.set_prf(prf24[centre-radius:centre+radius+1,centre-radius:centre+radius+1]/1.0E3,np.arange(0,41/2.0,0.5),np.arange(0,41/2.0,0.5))


In [33]:
print('fitting '+ str(prior250.nsrc)+' sources \n')
print('using ' +  str(prior250.snpix)+', '+ str(prior350.snpix)+' and '+ str(prior500.snpix)+' pixels')
print('source density = {}'.format(prior250.nsrc/moc.area_sq_deg))

fitting 35 sources 

using 835, 431 and 209 pixels
source density = 14441.029394120022


Before fitting, the prior classes need to take the PSF and calculate how muich each source contributes to each pixel. This process provides what we call a pointing matrix. Lets calculate the pointing matrix for each prior class

In [34]:
#prior24.get_pointing_matrix()
prior250.get_pointing_matrix()
prior350.get_pointing_matrix()
prior500.get_pointing_matrix()


In [35]:
xidplus.save([prior250,prior350,prior500],None,'./output/{}/prior_'.format(field[0])+esb['field'][source[0]]+'_'+str(source[0]))

In [36]:
prior_list.write('./output/{}/prior_list_'.format(field[0])+esb['field'][source[0]]+'_'+str(source[0])+'.ascii',format='ascii',overwrite=True)